In [1]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

import CMINE_lib as CMINE
# from Guassian_variables import Data_guassian
from structurerl import * 

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [2]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
from modules.CMI import DR_CMI, CDL_CMI

In [5]:
Dim = 5
batch_size = 64
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = create_dataset_DGP_binary_A_conf( Dim=5, N=batch_size)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

In [6]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [7]:
def train_dr(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=N)
        #dataset = create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()
        #print(a)

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values

In [8]:
def train_cdl(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=64)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return cdl_est_values

In [9]:
N=16
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.55841162e-01 1.83366506e-01 7.63569712e-01 1.81353542e-01
 1.91939004e+01 8.49449045e+01 1.08508483e+01 6.22935141e+01
 5.90143340e+04 9.97852150e+02]
[3.28495663e+00 1.92005296e+02 9.78379222e-01 9.20910714e-01
 3.18418048e+00 1.40984777e+05 7.62842189e+00 1.08530980e+01
 4.70197222e+02 5.52779821e+01]


In [10]:
np.random.randint(0, 1)

0

In [11]:
N = 128
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[3.71821213e+03 3.92298965e+03 3.92560532e+03 4.07484497e+03
 3.92057935e+03 1.81596002e+05 9.33726825e+04 3.76935484e+04
 9.85317383e+07 2.06142556e+06]
[4.63405831e+03 3.99647462e+03 3.83436325e+05 1.81859468e+03
 1.49838548e+07 1.83382114e+02 4.27130462e+05 4.74919770e+01
 4.08414217e+02 6.06603382e+01]


In [12]:
N = 32
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.93015250e+01 1.93217939e+01 1.93001622e+01 1.95151818e+01
 1.92402013e+01 2.59949862e+03 7.23254312e+04 4.48272244e+07
 5.50522303e+03 7.26690101e+02]
[6.97830773e+00 1.15795425e+04 3.91971305e+00 3.98680437e+00
 3.74568180e+00 2.13246227e+02 1.83963891e+06 2.04545174e+05
 3.31064171e+02 9.15712292e+03]


In [13]:
N = 64
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[58.36168671  0.61062402  3.47006393  0.20982889  1.38706867  0.33824272
  0.22131474  1.47916541  0.23722787  2.16484535]
[2.85136978e+01 1.37017854e+00 1.99884313e+01 7.38385294e+01
 2.44119127e+02 6.53696296e+03 4.12326024e+04 1.20587470e+05
 1.75584465e+06 2.07950426e+02]


# Training Step

In [26]:
N = 64
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[6.31943605e+04 1.71830261e+08 6.60621143e+04 1.83854909e+05
 6.29557521e+04 3.95083027e+09 5.30933075e+05 3.03847897e+05
 3.01877772e+06 9.41156254e+05]
[1.45674717e+05 5.62305784e+05 1.31564423e+05 1.34648689e+05
 1.32427251e+06 2.39606343e+09 1.46911331e+14 6.66713691e+09
 1.28531635e+08 2.40902858e+09]


In [27]:
N = 32
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.68653295e+04 8.02705649e+00 1.19907736e+03 3.14780780e+01
 3.98932782e+01 8.38730004e+05 3.92318122e+03 6.95953489e+04
 1.22019822e+05 3.48546747e+06]
[9.74688966e+03 7.74597234e+03 2.96815940e+09 6.31418542e+03
 6.20089638e+03 1.22671743e+06 3.68697284e+09 4.77083438e+06
 1.44713419e+07 1.88002237e+07]


In [31]:
N = 526
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.50006065e+01 2.18871596e+02 1.10105989e+07 5.11114454e+02
 1.06051950e+02 9.00031509e+01 1.04145438e+04 3.52512225e+03
 5.52125692e+03 2.41971664e+04]
[7.68950225e+06 7.74432824e+06 7.91731634e+06 7.70377130e+06
 7.70934032e+06 7.88427070e+08 4.68894041e+16 1.52884448e+10
 1.34091251e+11 2.40433729e+10]


In [30]:
N = 526
training_step = 10
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.41623130e-01 5.83663910e-01 2.62432172e-01 1.86141613e-01
 2.88988191e+02 2.00060656e+01 5.57397880e+00 2.69901884e+01
 2.13138771e-01 1.53580540e+00]
[1.34774063e+06 7.49846064e+03 7.87508115e+03 7.57209025e+03
 7.50388969e+03 5.57463735e+05 1.85607135e+04 3.51530271e+09
 1.84042864e+05 2.73516473e+06]
